In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import io
import cv2
from tqdm import tqdm_notebook as tqdm
import zipfile
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import keras
from keras import layers
from keras.applications import DenseNet201
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import gc

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tensorflow import keras
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import clone_model
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# let's start to analyse te each gievn csv
train_df=pd.read_csv("/kaggle/input/bengaliai-cv19/train.csv")

class_map=pd.read_csv("/kaggle/input/bengaliai-cv19/class_map.csv")


In [ ]:
train_df.head()


In [ ]:
class_map.groupby("component_type")["component"].nunique() # we have this much of image in each class


In [ ]:
class_map.head()

In [ ]:
import gc
gc.collect()


In [ ]:

data0 = pd.read_feather('/kaggle/input/bengali-preprocessed-dataset/train_data_0.feather')
data1 = pd.read_feather('/kaggle/input/bengali-preprocessed-dataset/train_data_1.feather')
data2 = pd.read_feather('/kaggle/input/bengali-preprocessed-dataset/train_data_2.feather')
data3 = pd.read_feather('/kaggle/input/bengali-preprocessed-dataset/train_data_3.feather')
data_full = pd.concat([data0,data1,data2,data3],ignore_index=True)
data = pd.merge(data_full, train_df, on='image_id').drop(['image_id'], axis=1).drop(['grapheme'], axis=1)

In [ ]:
data0.head()

In [ ]:
# delete all the data variable for memory issue be carefull.
del data0,data1,data2,data3,data_full

In [ ]:
data.shape

In [ ]:
HEIGHT = 64
WIDTH = 64

N_CHANNELS=1

# we can do for one class only (vowel_diacritic)


In [ ]:
del data["grapheme_root"]
del data["consonant_diacritic"]


In [ ]:
vowel_target=data["vowel_diacritic"]
del data["vowel_diacritic"]
x_train=data
del data

In [ ]:
def get_dummies(df):
    cols = []
    for col in df:
        cols.append(pd.get_dummies(df[col].astype(str)))
    return pd.concat(cols, axis=1)

In [ ]:
vowel_target=pd.get_dummies(vowel_target).values
type(vowel_target)


In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [ ]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction_vowel = ReduceLROnPlateau(monitor='loss', 
                                            patience=5, 
                                            verbose=1,
                                            factor=0.2, 
                                            )

In [ ]:
gc.collect()

In [ ]:
IMG_SIZE=64

In [ ]:
inputs = Input(shape = (IMG_SIZE, IMG_SIZE, 1))

model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1))(inputs)
model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.9)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
model = Dropout(rate=0.3)(model)

model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.9)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=64, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.9)(model)
model = Dropout(rate=0.3)(model)

model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.9)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=128, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.9)(model)
model = Dropout(rate=0.3)(model)

model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.9)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=256, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.9)(model)
model = Dropout(rate=0.3)(model)

model = Flatten()(model)
model = Dense(1024, activation = "relu")(model)
model = Dropout(rate=0.3)(model)
dense = Dense(512, activation = "relu")(model)
head_vowel = Dense(11, activation = 'softmax')(dense)
model = Model(inputs=inputs, outputs=[head_vowel])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
x_train.shape

In [ ]:
x_train.head()

In [ ]:
x_train = x_train.values.reshape(-1,64,64,1)


In [ ]:
print(vowel_target.shape,x_train.shape)

In [ ]:
datagen.fit(x_train)

In [ ]:
batch_size=128
epochs=100


In [ ]:
# Fit the model
history = model.fit_generator(datagen.flow(x_train,vowel_target , batch_size=batch_size),
                          epochs = epochs,
                          steps_per_epoch=x_train.shape[0] // batch_size, 
                          callbacks=[learning_rate_reduction_vowel])
gc.collect()

In [ ]:
%matplotlib inline
def plot_loss(his, epoch, title):
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, epoch), his.history['loss'], label='train_loss')
    plt.title(title)
    plt.xlabel('Epoch #')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.show()

def plot_acc(his, epoch, title):
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, epoch), his.history['accuracy'], label='train_vowel_loss')
    plt.title(title)
    plt.xlabel('Epoch #')
    plt.ylabel('Accuracy')
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
plot_loss(history, epochs, f'Training Loss')
plot_acc(history, epochs, f'Training Accuracy')

In [ ]:
model.save('simple_cnn_vowel_100_epoch.h5')